In [28]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

from scripts.preprocessing import NanHandler, TrendHandler, OkvedHandler, ColumnRemover, InfoHandler

In [17]:
train_df = pd.read_parquet('data/train.parquet')
test_df = pd.read_parquet('data/test.parquet')

In [18]:
### Preprocessing config ###

# Running mode 
split_train = True # whether 'train_df' will be splitted on train and val during current run. Should not be splitted if submittion is expected futher.  

# Nan handling 
del_high_miss_cols = True # Whether to delete columns with high percentage of missing values
impute_strategy = None  # None - do not impute, 'avg' - simple imputer (implemented), 'kmeans' - kmeans imputer (not implemented), 'iterative' - iteratieve imputer (not implemented)
update_nans = True # Whether to translate values from NanHandler.convert_to_na_dict to np.nan i.e. replace -999 in rko_start_months to np.nan 

# Trend generator
apply_trend_gen = True # Whether to apply trend handler to data (if False, options below will be ignored) -> generates columns '{x}_trend' for x in [unique prefixes of columns that end with _1m, _3m, _1y ...] 
drop_old_trend_cols = True  # Whether to drop columns out of which trend was calculated i.e. sum_deb_e_oper_1m, cnt_deb_e_oper_1m 

# Okved handling
apply_okved_transform = True # Whether to apply okved handler to data (if False, options below will be ignored) -> generates column 'okved_groups'
drop_old_okved = True # Whether to drop original 'okved' column
apply_okved_ohe = False # Whether to apply one hot encoding to new transformed column 'okved_groups'


col_remover = ColumnRemover()
coltype_handler = InfoHandler()
nan_handler = NanHandler(del_high_miss_cols, impute_strategy, update_nans)
trend_handler = TrendHandler(apply_handler=apply_trend_gen, drop_old_cols=drop_old_trend_cols)
okved_handler = OkvedHandler(drop_old_col=drop_old_okved, apply_ohe=apply_okved_ohe, apply_handler=apply_okved_transform)

In [19]:
col_remover.process(train_df)
trend_handler.process(train_df)
okved_handler.process(train_df)

col_remover.process(test_df)
trend_handler.process(test_df)
okved_handler.process(test_df)

[ColumnRemover] Dropped columns ['id']


[TrendHandler] Calculating m1 trends...: 100%|██████████| 1/1 [00:00<00:00, 18.52it/s]


[TrendHandler] Dropped old columns: ['sum_a_oper_1m', 'cnt_a_oper_1m', 'sum_b_oper_1m', 'cnt_b_oper_1m', 'sum_c_oper_1m', 'cnt_c_oper_1m', 'sum_deb_d_oper_1m', 'cnt_deb_d_oper_1m', 'sum_deb_e_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m', 'sum_cred_e_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m', 'sum_deb_f_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m', 'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m', 'sum_deb_g_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_days_deb_g_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_1m', 'sum_deb_h_oper_1m', 'cnt_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'sum_cred_h_oper_1m', 'cnt_cred_h_oper_1m', 'cnt_days_cred_h_oper_1m', 'sum_a_oper_3m', 'cnt_a_oper_3m', 'sum_b_oper_3m', 'cnt_b_oper_3m', 'sum_c_oper_3m', 'cnt_c_oper_3m', 'sum_deb_d_oper_3m', 'cnt_deb_d_oper_3m', 'sum_deb_e_oper_3m', 'cnt_deb_e_oper_3m', 'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m', 'cnt_cred_e_oper_3m', 'cnt_da

[TrendHandler] Calculating m1 trends...: 100%|██████████| 1/1 [00:00<00:00, 83.33it/s]


[TrendHandler] Dropped old columns: ['sum_a_oper_1m', 'cnt_a_oper_1m', 'sum_b_oper_1m', 'cnt_b_oper_1m', 'sum_c_oper_1m', 'cnt_c_oper_1m', 'sum_deb_d_oper_1m', 'cnt_deb_d_oper_1m', 'sum_deb_e_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m', 'sum_cred_e_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m', 'sum_deb_f_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m', 'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m', 'sum_deb_g_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_days_deb_g_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_1m', 'sum_deb_h_oper_1m', 'cnt_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'sum_cred_h_oper_1m', 'cnt_cred_h_oper_1m', 'cnt_days_cred_h_oper_1m', 'sum_a_oper_3m', 'cnt_a_oper_3m', 'sum_b_oper_3m', 'cnt_b_oper_3m', 'sum_c_oper_3m', 'cnt_c_oper_3m', 'sum_deb_d_oper_3m', 'cnt_deb_d_oper_3m', 'sum_deb_e_oper_3m', 'cnt_deb_e_oper_3m', 'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m', 'cnt_cred_e_oper_3m', 'cnt_da

In [20]:
if split_train:
    train_indices, val_indices = train_test_split(list(train_df.index), stratify=train_df['total_target'], test_size=0.2, random_state=42)
    val_df = train_df.iloc[val_indices]
    train_df = train_df.iloc[train_indices]
    X_train, y1_train, y2_train, y_train = train_df.drop(['target_1', 'target_2', 'total_target'], axis=1), train_df['target_1'], train_df['target_2'], train_df['total_target']
    X_val, y1_val, y2_val, y_val = val_df.drop(['target_1', 'target_2', 'total_target'], axis=1), val_df['target_1'], val_df['target_2'], val_df['total_target']
else:
    X_train, y1_train, y2_train, y_train = train_df.drop(['target_1', 'target_2', 'total_target'], axis=1), train_df['target_1'], train_df['target_2'], train_df['total_target']

X_test = test_df

In [21]:
nan_handler.process(X_train, is_train=True)
nan_handler.process(X_val, is_train=False)
nan_handler.process(X_test, is_train=False)

[NanHandler] Deleting high percentage nan columns: ['max_end_plan_non_fin_deals', 'max_start_non_fin_deals', 'min_start_non_fin_deals', 'min_end_plan_non_fin_deals', 'max_start_fin_deals', 'min_end_fact_fin_deals', 'min_start_fin_deals', 'max_end_fact_fin_deals']
[NanHandler] Converted value -999 from column rko_start_months to np.nan
[NanHandler] Deleting high percentage nan columns: ['max_end_plan_non_fin_deals', 'max_start_non_fin_deals', 'min_start_non_fin_deals', 'min_end_plan_non_fin_deals', 'max_start_fin_deals', 'min_end_fact_fin_deals', 'min_start_fin_deals', 'max_end_fact_fin_deals']
[NanHandler] Converted value -999 from column rko_start_months to np.nan
[NanHandler] Deleting high percentage nan columns: ['max_end_plan_non_fin_deals', 'max_start_non_fin_deals', 'min_start_non_fin_deals', 'min_end_plan_non_fin_deals', 'max_start_fin_deals', 'min_end_fact_fin_deals', 'min_start_fin_deals', 'max_end_fact_fin_deals']
[NanHandler] Converted value -999 from column rko_start_months

In [25]:
col_types = InfoHandler.get_cols_info(X_train)
categorical_columns = col_types['categorical_cols']
numerical_columns = col_types['numerical_cols']

In [26]:
X_train[categorical_columns] = X_train[categorical_columns].astype("category")
X_val[categorical_columns] = X_val[categorical_columns].astype("category")
X_test[categorical_columns] = X_test[categorical_columns].astype("category")

In [30]:
model = LGBMClassifier(verbose=-1)
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]
print(f"ROC-AUC: {roc_auc_score(y_val, y_val_proba)}")
print(classification_report(y_val, y_val_pred))

ROC-AUC: 0.8821342177202827
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     66937
           1       0.63      0.24      0.35      5063

    accuracy                           0.94     72000
   macro avg       0.79      0.62      0.66     72000
weighted avg       0.92      0.94      0.92     72000

